In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]='0,1,2,3'
import sys
import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('-mode', type=str, help='rgb or flow')
# parser.add_argument('-save_model', type=str)
# parser.add_argument('-root', type=str)

# args = parser.parse_args()


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

import torchvision
from torchvision import datasets, transforms
import videotransforms


import numpy as np

from pytorch_i3d import InceptionI3d

# from charades_dataset import Charades as Dataset
from visual_tactile_dataset import VisualTactile as Dataset
import cv2
import matplotlib.pyplot as plt

In [ ]:
path = "/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/"
i3d = InceptionI3d(400, in_channels=2)
i3d.load_state_dict(torch.load('models/flow_imagenet.pt'))
i3d.train(True)

train_transforms = transforms.Compose([videotransforms.RandomCrop(224),
                                       videotransforms.RandomHorizontalFlip(),])
# train_transforms = transforms.Compose([videotransforms.CenterCrop(480),
#                                        videotransforms.RandomHorizontalFlip(),])
dataset = Dataset('train.txt',path, transform=train_transforms)
# dataset = Dataset(path)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0, pin_memory=False)
# for data in dataset:
c = 0
for data in dataloader:
    print("ok getting single data")
    f_vid, l_vid, tactile, pos, label = data
    c += 1
    if (c%2 == 0):
        break

# image,tactile,control_label,front_video,left_video,pos,label = dataset[0]

In [ ]:
print(f_vid.shape)
plt.imshow(f_vid[0][0][0])
# torch.from_numpy(left_video.transpose([3,0,1,2])).size()
# np.array(front_video[0]).shape

In [ ]:
path = "/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/Cheez/50_432/top/0/front_0.jpg"
image = cv2.imread(path)
# cv2.imshow('im', image)
# image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
# cv2.imshow('im',image)
# cv2.waitKey(1000)
np.asarray(image, dtype=np.float32)


In [ ]:
train_transforms = transforms.Compose([videotransforms.RandomCrop(224),
                                       videotransforms.RandomHorizontalFlip(),])

In [ ]:
def run(init_lr=0.1, max_steps=64e3, mode='rgb', root="/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/"
        , train_split=None, batch_size=1, save_model=''):
#     # setup dataset
#     train_transforms = transforms.Compose([videotransforms.RandomCrop(224),
#                                            videotransforms.RandomHorizontalFlip(),
#     ])
#     test_transforms = transforms.Compose([videotransforms.CenterCrop(224)])

    dataset = Dataset(root)
    dataset = Dataset(root,transform=videotransforms.CenterCrop(224))
#     dataset = Dataset(train_split, 'training', root, mode, train_transforms)
#     dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=36, pin_memory=True)

#     val_dataset = Dataset(train_split, 'testing', root, mode, test_transforms)
#     val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=36, pin_memory=True)    

#     dataloaders = {'train': dataloader, 'val': val_dataloader}
#     datasets = {'train': dataset, 'val': val_dataset}

    
    # setup the model
    if mode == 'flow':
        i3d = InceptionI3d(400, in_channels=2)
        i3d.load_state_dict(torch.load('models/flow_imagenet.pt'))
    else:
        i3d = InceptionI3d(400, in_channels=3)
        i3d.load_state_dict(torch.load('models/rgb_imagenet.pt'))
    i3d.replace_logits(157)
#     #i3d.load_state_dict(torch.load('/ssd/models/000920.pt'))
#     i3d.cuda()
#     i3d = nn.DataParallel(i3d)

    lr = init_lr
    optimizer = optim.SGD(i3d.parameters(), lr=lr, momentum=0.9, weight_decay=0.0000001)
    lr_sched = optim.lr_scheduler.MultiStepLR(optimizer, [300, 1000])

    num_steps_per_update = 4 # accum gradient
    steps = 0
    # train it
    while steps < max_steps:#for epoch in range(num_epochs):
        print ('Step {}/{}'.format(steps, max_steps))
        print ('-' * 10)

#         # Each epoch has a training and validation phase
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 i3d.train(True)
#             else:
#                 i3d.train(False)  # Set model to evaluate mode
        i3d.train(True)
        tot_loss = 0.0
        tot_loc_loss = 0.0
        tot_cls_loss = 0.0
        num_iter = 0
        optimizer.zero_grad()

        # Iterate over data.
#         for data in dataloaders[phase]:
        for data in dataset:
            num_iter += 1
            # get the inputs
#             inputs, labels = data
            image,tactile,control_label,front_video,left_video,pos,label = data

#             # wrap them in Variable
#             inputs = Variable(inputs.cuda())
#             t = inputs.size(2)
#             labels = Variable(labels.cuda())
            front_video = front_video.unsqueeze(0)
            inputs = Variable(front_video)
#             inputs = torch.FloatTensor(front_video)
#             inputs = Variable(inputs)
            print(type(inputs))
            print(inputs.size())
#             inputs = front_video
            t = len(inputs)
            print(inputs.size())
            
            print("calculationg per_frame_logits")
            per_frame_logits = i3d(inputs)
            # upsample to input size
            print("upsampling per_frame_logits", per_frame_logits)
            per_frame_logits = F.upsample(per_frame_logits, t, mode='linear')
            F.interpolate(align_corners=False)

            print("computing localization loss")
            # compute localization loss
            loc_loss = F.binary_cross_entropy_with_logits(per_frame_logits, labels)
            tot_loc_loss += loc_loss.data[0]

            # compute classification loss (with max-pooling along time B x C x T)
            cls_loss = F.binary_cross_entropy_with_logits(torch.max(per_frame_logits, dim=2)[0], torch.max(labels, dim=2)[0])
            tot_cls_loss += cls_loss.data[0]
            loss = (0.5*loc_loss + 0.5*cls_loss)/num_steps_per_update
            tot_loss += loss.data[0]
            loss.backward()

            print ('{} Loc Loss: {:.4f} Cls Loss: {:.4f} Tot Loss: {:.4f}'.format(phase, tot_loc_loss/(10*num_steps_per_update), tot_cls_loss/(10*num_steps_per_update), tot_loss/10))
            if num_iter == num_steps_per_update and phase == 'train':
                steps += 1
                num_iter = 0
                optimizer.step()
                optimizer.zero_grad()
                lr_sched.step()
                if steps % 10 == 0:
                    print ('{} Loc Loss: {:.4f} Cls Loss: {:.4f} Tot Loss: {:.4f}'.format(phase, tot_loc_loss/(10*num_steps_per_update), tot_cls_loss/(10*num_steps_per_update), tot_loss/10))
                    # save model
                    torch.save(i3d.module.state_dict(), save_model+str(steps).zfill(6)+'.pt')
                    tot_loss = tot_loc_loss = tot_cls_loss = 0.
        if phase == 'val':
            print ('{} Loc Loss: {:.4f} Cls Loss: {:.4f} Tot Loss: {:.4f}'.format(phase, tot_loc_loss/num_iter, tot_cls_loss/num_iter, (tot_loss*num_steps_per_update)/num_iter) )
    


if __name__ == '__main__':
    # need to add argparse
#     run(mode=args.mode, root=args.root, save_model=args.save_model)
    run()


# Architecture after modified dataloader.

In [ ]:
# def run(init_lr=0.1, max_steps=64e3, mode='rgb', root='/ssd/Charades_v1_rgb', train_split='charades/charades.json', batch_size=8*5, save_model=''):
def run(init_lr=0.001, max_steps=10, mode='rgb', root='/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/',\
        train_split='train.txt', test_split='test.txt', batch_size=1, save_model=''):
    # setup dataset
    train_transforms = transforms.Compose([videotransforms.RandomCrop(224),
                                           videotransforms.RandomHorizontalFlip(),])
    test_transforms = transforms.Compose([videotransforms.CenterCrop(224)])

    dataset = Dataset(train_split, root, mode, train_transforms)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

    val_dataset = Dataset(test_split, root, mode, test_transforms)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)    

    dataloaders = {'train': dataloader, 'val': val_dataloader}
    datasets = {'train': dataset, 'val': val_dataset}

    
    # setup the model
    if mode == 'flow':
        i3d = InceptionI3d(400, in_channels=2)
        i3d.load_state_dict(torch.load('models/flow_imagenet.pt'))
    else:
        i3d = InceptionI3d(400, in_channels=3)
        i3d.load_state_dict(torch.load('models/rgb_imagenet.pt'))
    i3d.replace_logits(157)
#     #i3d.load_state_dict(torch.load('/ssd/models/000920.pt'))
#     i3d.cuda()
#     i3d = nn.DataParallel(i3d)

    lr = init_lr
    optimizer = optim.SGD(i3d.parameters(), lr=lr, momentum=0.9, weight_decay=0.0000001)
#     lr_sched = optim.lr_scheduler.MultiStepLR(optimizer, [300, 1000])
    lr_sched = optim.lr_scheduler.MultiStepLR(optimizer, [4, 7])


    num_steps_per_update = 4 # accum gradient
    steps = 0
    # train it
    while steps < max_steps:#for epoch in range(num_epochs):
        print ('Step {}/{}'.format(steps, max_steps))
        print ('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                i3d.train(True)
            else:
                i3d.train(False)  # Set model to evaluate mode
                
            tot_loss = 0.0
            tot_loc_loss = 0.0
            tot_cls_loss = 0.0
            num_iter = 0
            optimizer.zero_grad()
            
            # Iterate over data.
            for data in dataloaders[phase]:
                num_iter += 1
                # get the inputs
                f_vid, l_vid, tactile, pos, labels = data

                # wrap them in Variable
                inputs = Variable(f_vid)
                t = inputs.size(2)
                labels = Variable(labels)
#                 inputs = Variable(inputs.cuda())
#                 t = inputs.size(2)
#                 labels = Variable(labels.cuda())
                print("go to i3d")
                per_frame_logits = i3d(inputs.float())
                # upsample to input size
                per_frame_logits = F.upsample(per_frame_logits, t, mode='linear')

                # compute localization loss
                loc_loss = F.binary_cross_entropy_with_logits(per_frame_logits, labels)
                tot_loc_loss += loc_loss.data[0]

                # compute classification loss (with max-pooling along time B x C x T)
                cls_loss = F.binary_cross_entropy_with_logits(torch.max(per_frame_logits, dim=2)[0], torch.max(labels, dim=2)[0])
                tot_cls_loss += cls_loss.data[0]

                loss = (0.5*loc_loss + 0.5*cls_loss)/num_steps_per_update
                tot_loss += loss.data[0]
                loss.backward()

                if num_iter == num_steps_per_update and phase == 'train':
                    steps += 1
                    num_iter = 0
                    optimizer.step()
                    optimizer.zero_grad()
                    lr_sched.step()
                    if steps % 10 == 0:
                        print ('{} Loc Loss: {:.4f} Cls Loss: {:.4f} Tot Loss: {:.4f}'.format(phase, \
                            tot_loc_loss/(10*num_steps_per_update), tot_cls_loss/(10*num_steps_per_update), tot_loss/10))
                        # save model
                        torch.save(i3d.module.state_dict(), save_model+str(steps).zfill(6)+'.pt')
                        tot_loss = tot_loc_loss = tot_cls_loss = 0.
            if phase == 'val':
                print ('{} Loc Loss: {:.4f} Cls Loss: {:.4f} Tot Loss: {:.4f}'.format(phase, tot_loc_loss/num_iter, tot_cls_loss/num_iter, (tot_loss*num_steps_per_update)/num_iter) )
    


if __name__ == '__main__':
    # need to add argparse
#     run(mode=args.mode, root=args.root, save_model=args.save_model)
    run(save_model="test.pt")


# References
* https://discuss.pytorch.org/t/runtimeerror-expected-4-dimensional-input-for-4-dimensional-weight-6-3-5-5-but-got-3-dimensional-input-of-size-3-256-256-instead/37189/4
* https://medium.com/@zhang_yang/how-is-pytorchs-binary-cross-entropy-with-logits-function-related-to-sigmoid-and-d3bd8fb080e7

In [50]:
batch_size, n_classes = 5, 157
x = torch.randn(batch_size, n_classes,64)
y = torch.randn(5)

print(x.shape)
print(y.shape)

torch.Size([5, 157, 64])
torch.Size([5])


In [43]:
criterion = nn.CrossEntropyLoss()
criterion(torch.max(x, dim=2)[0],y)

RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target'

In [10]:
batch_size = 5
nb_classes = 10
model = nn.Linear(20, nb_classes)

x = torch.randn(batch_size, 20)
target = torch.empty(batch_size, dtype=torch.long).random_(nb_classes)

criterion = nn.CrossEntropyLoss()

output = model(x)
loss = criterion(output, target)

In [7]:
output.size()

torch.Size([5, 10])

In [32]:
criterion = nn.BCEWithLogitsLoss()
# criterion(torch.max(x, dim=2)[0],torch.max(y, dim=2)[0])
torch.max(x, dim=2)[0].shape
# x.shape

torch.Size([5, 157])

In [58]:
batch_size, n_classes = 5, 1
x = torch.randn(batch_size, n_classes,64)
# print(x)
x = torch.ones_like(x)
y = torch.randn(batch_size)
y = torch.zeros_like(y)
# F.binary_cross_entropy_with_logits(torch.max(x, dim=2)[0], y)

In [59]:
# y[4] = 0
print(y)
x.size()


tensor([0., 0., 0., 0., 0.])


torch.Size([5, 1, 64])

In [56]:
x = torch.max(x, dim=2)[0]
x

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [53]:
x = x.squeeze(1)
F.binary_cross_entropy_with_logits(x,y)


tensor(1.3133)

In [28]:
y

tensor([1., 1., 1., 1., 1.])

In [83]:
target = torch.randint(n_classes, size=(batch_size,), dtype=torch.long)
print(target)
y = torch.zeros(batch_size, n_classes)
y[range(y.shape[0]), target]=1
y

tensor([0, 1, 0, 0, 1])


tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.]])

In [87]:
F.binary_cross_entropy_with_logits(torch.max(x, dim=2)[0], y)

tensor(1.2780)

In [81]:
i3d = InceptionI3d(400, in_channels=2)
layers = []
for param in i3d.Conv3d_1a_7x7.parameters():
    layers.append(param.requires_grad)
len(layers)

3

In [85]:
i3d = InceptionI3d(400, in_channels=2)
layers = []
for param in i3d..parameters():
    layers.append(param.requires_grad)
len(layers)

0

In [104]:
y = torch.randn(5)
y[range(y.shape),y] = 1
y

TypeError: 'torch.Size' object cannot be interpreted as an integer

In [153]:
p = x

In [154]:
x.size()

torch.Size([5, 1, 1])

In [155]:
p = p.squeeze(2)

In [160]:
p = p.squeeze(1)
p.size()

torch.Size([5])

In [137]:
nn.CrossMapLRN2d(x,y)

CrossMapLRN2d(
  tensor([[-0.4089,  0.5265],
          [-0.6818, -0.8182],
          [ 1.4865,  0.2778],
          [-0.2442,  0.0745],
          [ 0.7655, -0.9072]]), alpha=tensor([ 0.2480,  0.2186,  1.0653, -1.2147,  0.5097]), beta=0.75, k=1
)

In [164]:
loss = F.binary_cross_entropy_with_logits(p,y)

In [62]:
loss = 10

In [65]:
with open("ok.txt", 'w') as file:
    file.write("%f \n" %(loss/10))

In [1]:
x = torch.randn(5, 1024 n_classes,64)


SyntaxError: invalid syntax (<ipython-input-1-0741528698cf>, line 1)